In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, GRU, Dense,Dropout
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS

In [ ]:
combined_df=pd.read_csv('/content/drive/MyDrive/combined.csv')

In [ ]:
combined_df['class'].value_counts()

1    900000
0    900000
Name: class, dtype: int64

In [ ]:
X = combined_df['reviewText'].values
y = combined_df['class'].values

In [ ]:
def preprocess_text(text):
    # Remove stopwords
    cleaned_text = ' '.join([word for word in text.split() if word.lower() not in ENGLISH_STOP_WORDS])
    # Clean the text further as needed (e.g., removing punctuation, lowering, stemming, etc.)
    cleaned_text = cleaned_text.lower()
    # Return cleaned text
    return cleaned_text


In [ ]:
X = [preprocess_text(text) for text in X]

In [ ]:
# Tokenization
max_words = 10000
max_len = 100
tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(X)
X = tokenizer.texts_to_sequences(X)
X= pad_sequences(X, maxlen=max_len)

In [ ]:
# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [ ]:
# Define your model
embedding_dim = 50  # Dimension of the word embeddings
gru_units = 64  # Number of units in the GRU layer

model = Sequential([
    Embedding(input_dim=len(tokenizer.word_index)+1,output_dim=128, input_length=max_len),
    GRU(gru_units, return_sequences=True),
    GRU(gru_units),
    Dense(64, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')
])

In [ ]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, y_train, epochs=5, batch_size=64, validation_data=(X_test, y_test))

Epoch 1/5
 3365/22500 [===>..........................] - ETA: 8:31:34 - loss: 0.3847 - accuracy: 0.8297